In [2]:
import numpy as np
import pandas as panda
features = ["utterance","frame","coefficient1","coefficient2","coefficient3","coefficient4","coefficient5","coefficient6","coefficient7","coefficient8","coefficient9","coefficient10","coefficient11","coefficient12","binaryClass"]
dosya = panda.read_csv("C:/Users/canimaginerc/Desktop/kdd_JapaneseVowels.csv", skiprows=1, names=features)
dosya = dosya.sample(frac=1).reset_index(drop=True)

#####################################################################################
#     CFG       CFG       CFG       CFG       CFG       CFG       CFG       CFG     #
#####################################################################################
hiddenNeuron = 8                                                                    #
learning_rate = 0.03                                                                #
epoch = 5000                                                                        #
trainingSize = 1500                                                                 #
###Do not touch below                                                               #
inputRows = 14                                                                      #
output = 1                                                                          #
overfit = 0                                                                         #
#####################################################################################
trainO, testO, traini = [], [], []
# Tahmin sınıflandırması için; "N"-0'lar için 0'dan en uzağı bulmak için "fMax" ile, "P"-1'ler için 1'den en uzağı bulmak için de "tMax kullanacağız
FP, TP, FN, TN, fMax, tMax = 0, 0, 0, 0, 0, 1
#W1, W2, B1, B2 = 0, 0, 0, 0

################################################################
def validation(x):
    valSets = [1, 2, 3, 4, 5] # We call a number which choosen for test, other part of validations gets union.
    valSets.remove(x)
    
    #Test Set Determines Here
    testInput = dosya.iloc[((x-1)*1992):(x*1992), 0:14]
    testOutput = dosya.iloc[((x-1)*1992):(x*1992), 14:15]
    
    
    #Training Validation Sets Gets Union Here
    setIndex = valSets[0]
    inputSet = dosya.iloc[((setIndex-1)*1992):(setIndex*1992), 0:14]
    outputSet = dosya.iloc[((setIndex-1)*1992):(setIndex*1992), 14:15]
    for i in range(1,4):
        setIndex = valSets[i]
        tmpInputSet = dosya.iloc[((setIndex-1)*1992):(setIndex*1992), 0:14]
        inputSet = inputSet.append(tmpInputSet, ignore_index = True)
        tmpOutputSet = dosya.iloc[((setIndex-1)*1992):(setIndex*1992), 14:15]
        outputSet = outputSet.append(tmpOutputSet, ignore_index = True)
    return(inputSet, outputSet, testInput, testOutput)

#It's change to OutputDF's (train and test -both of it) to "0-1" matrices
def dftomatrix(trainSDF, trainDF, testDF):
    inputM, trDFm, teDFm, tmpi = [], [], [], []    
    
    #Input DataFrame to Matrix
    for i in range(7968):
        tmpi = trainSDF.values[i]
        inputM.append(tmpi)

    #Train Output DF and Test Output DF to Matrices
    for i in range(1992):
        if trainDF.values[i] == "N":
            trDFm.append(0)
        else:
            trDFm.append(1)

    for i in range(1992):
        if testDF.values[i] == "N":
            teDFm.append(0)
        else:
            teDFm.append(1)
    return (inputM, trDFm, teDFm)



def randoom():
    np.random.seed(1)
    W1 = np.random.normal(0, 1, (hiddenNeuron, inputRows))
    W2 = np.random.normal(0, 1, (output, hiddenNeuron))

    B1 = np.random.random((hiddenNeuron, 1))
    B2 = np.random.random((output, 1))
    return (W1, W2, B1, B2)

def sigmoid(z, derv=False):
    if derv: return z * (1 - z)
    return 1 / (1 + np.exp(-z))


def forward(x, predict=False):
    a1 = x.reshape(x.shape[0], 1)

    z2 = W1.dot(a1) + B1
    a2 = sigmoid(z2)

    z3 = W2.dot(a2) + B2
    a3 = sigmoid(z3)

    if predict: return a3
    return (a1, a2, a3)


dW1 = 0
dW2 = 0

dB1 = 0
dB2 = 0


def train(X, y, tW1, tW2, tB1, tB2):
    for i in range(epoch):
        
        dW1 = 0
        dW2 = 0

        dB1 = 0
        dB2 = 0
        
        for j in range(trainingSize):

            # Forward
            a0 = X[j].reshape(X[j].shape[0], 1)

            z1 = tW1.dot(a0) + tB1
            a1 = sigmoid(z1)

            z2 = tW2.dot(a1) + tB2
            a2 = sigmoid(z2)

            # Back
            dz2 = a2 - y[j]
            dW2 += dz2 * a1.T

            dz1 = np.multiply((tW2.T * dz2), sigmoid(a1, derv=True))
            dW1 += dz1.dot(a0.T)

            dB1 += dz1
            dB2 += dz2

        
        tW1 = tW1 - learning_rate * (dW1 / trainingSize) + ( (overfit / trainingSize) * tW1)
        tW2 = tW2 - learning_rate * (dW2 / trainingSize) + ( (overfit / trainingSize) * tW2)

        tB1 = tB1 - learning_rate * (dB1 / trainingSize)
        tB2 = tB2 - learning_rate * (dB2 / trainingSize)
    return (tW1, tW2, tB1, tB2)
#####################################################################################################

def testFunc(FP, TP, FN, TN, tesT, tesTyC):
    fMax, tMax = 0, 1
    for i in range(1992):
        deger = forward(tesT.values[i], predict=True)
        if tesTyC[i] == 0:
            if deger  > 0.31:
                FP += 1
                if deger > fMax:
                    fMax = deger
            else:
                TN += 1
                if deger > fMax:
                    fMax = deger
        

        elif tesTyC[i] == 1:
            if deger <= 0.31:
                FN += 1
                if deger < tMax:
                    tMax = deger
            else:
                TP += 1
                if deger < tMax:
                    tMax = deger
   # These rows used for get information while trying to find optimal values
   # print(str(fMax) + "  --  " + str(tMax))
   # print("TP: " + str(TP) + "    TN: " + str(TN) +"\n" + "FP: " + str(FP) + "    FN: " + str(FN))
    
    return (FP, TP, FN, TN, fMax, tMax)
 #Calculation Functions   
#####################################################################################################
def reCallP (FP, TP, FN, TN):
    if TP == 0:
        return (0.01)
    else:
        return (TP/(TP + FN))
def preCisionP (FP, TP, FN, TN):
    if TP == 0:
        return (0.01)
    else:
        return (TP/(TP + FP))
def fScoreP (FP, TP, FN, TN):
    if TP == 0:
        return (0.01)
    else:
        return (2*((preCisionP(FP, TP, FN, TN)*reCallP(FP, TP, FN, TN))/(preCisionP(FP, TP, FN, TN)+reCallP(FP, TP, FN, TN))))
def averageRecallMacro(avReMa):
    aRMS = np.matrix(avReMa)
    aRMS = aRMS.sum()
    aRM = aRMS / len(avReMa)
    return (aRM)
    
def averageRecallMicro(TPS, FNS):
    trpos =  np.matrix(TPS)
    trpos = trpos.sum()
    faneg = np.matrix(FNS)
    faneg = faneg.sum()
    avReMi = trpos / (trpos + faneg)
    return (avReMi)
    
def averagePrecisionMacro(avPrMa):
    aPMS = np.matrix(avPrMa)
    aPMS = aPMS.sum()
    aPM = aPMS / len(avPrMa)
    return (aPM)
def averagePrecisionMicro(TPM, FPM):
    trposs =  np.matrix(TPM)
    trposs = trposs.sum()
    fapos = np.matrix(FPM)
    fapos = fapos.sum()
    avRrMi = trposs / (trposs + fapos)
    return (avRrMi)

#####################################################################################################
FPs, TPs, FNs, TNs, recalls, precisions = [], [], [], [], [], []
for i in range(1,6): #Validation Loop
    FP, TP, FN, TN, fMax, tMax = 0, 0, 0, 0, 0, 1

    trainSet, trainOut, testInput, testOutput = validation(i)

    traini, trainO, testO = dftomatrix(trainSet,  trainOut, testOutput)

    W1, W2, B1, B2 = randoom()

    W1, W2, B1, B2 = train(traini, trainO, W1, W2, B1, B2)

    FP, TP, FN, TN, fMax, tMax = testFunc(FP, TP, FN, TN, testInput, testO)
    
    FPs.append(FP)
    TPs.append(TP)
    FNs.append(FN)
    TNs.append(TN)
    recalls.append(reCallP(FP, TP, FN, TN))
    precisions.append(preCisionP(FP, TP, FN, TN))

    print("Scores for Class: " + str(i) + "\n ===============================")
    print("Recall -P: " + str(reCallP(FP, TP, FN, TN)))
    print("Precision -P: " + str(preCisionP(FP, TP, FN, TN)))
    print("F-Score -P: " + str(fScoreP(FP, TP, FN, TN)) + "\n")

print("\nTotal Scores:  \n")
print("Average Recall -Macro: " + str(averageRecallMacro(recalls)))    
print("Average Recall -Micro: " + str(averageRecallMicro(TPs, FNs)))
print("Average F-Score -Macro: " + str((2*((averageRecallMacro(recalls)*averagePrecisionMacro(precisions))/(averageRecallMacro(recalls)+averagePrecisionMacro(precisions))))))
print("Average F-Score -Micro: " + str((2*((averagePrecisionMicro(TPs, FPs)*averageRecallMicro(TPs, FNs))/(averagePrecisionMicro(TPs, FPs)+averageRecallMicro(TPs, FNs))))))      
print("END")

Scores for Class: 1
Recall -P: 0.5623003194888179
Precision -P: 0.7213114754098361
F-Score -P: 0.6319569120287253

Scores for Class: 2
Recall -P: 0.6480263157894737
Precision -P: 0.7060931899641577
F-Score -P: 0.6758147512864494

Scores for Class: 3
Recall -P: 0.6228571428571429
Precision -P: 0.6707692307692308
F-Score -P: 0.645925925925926

Scores for Class: 4
Recall -P: 0.6296296296296297
Precision -P: 0.6559485530546624
F-Score -P: 0.6425196850393701

Scores for Class: 5
Recall -P: 0.6718266253869969
Precision -P: 0.7161716171617162
F-Score -P: 0.6932907348242812


Total Scores:  

Average Recall -Macro: 0.6269280066304121
Average Recall -Micro: 0.6270136307311028
Average F-Score -Macro: 0.6587876604567571
Average F-Score -Micro: 0.6579973992197659
END
